### Tensorflow model is not complete, As the deadline was over

In [2]:
import tensorflow as tf
import collections
import numpy as np

#layers = tf.contrib.layers
print(tf.__version__)

1.12.0


In [3]:
from gensim.models.wrappers import FastText
fasttxt_model = FastText.load_fasttext_format('/mnt/069A453E9A452B8D/Ram/Downloads/cc.en.300.bin')

In [4]:
print("fastxt model similaritie of words:")
print("fraud, froud =>", fasttxt_model.similarity('fraud', 'froud'))
print("awesome, awsm =>", fasttxt_model.similarity('awesome', 'awsm'))
print("awesome, good =>", fasttxt_model.similarity('awesome', 'good'))

fastxt model similaritie of words:
fraud, froud => 0.39351234
awesome, awsm => 0.42913762
awesome, good => 0.5001332


In [21]:
# Parameters

# preporcessed files with words replaced with voc index
train_csv_path = "train.csv"
val_csv_path = ""
voc_file = ""
summary_dir = ""

vocab_size = 10000
emb_size = 300


NUM_BI_LAYER = 1
NUM_HIDDEN = 100
NUM_CLASSES = 5
batch_size = 4
learning_rate = 0.001
max_iter = 1

### Importing csv dataset

In [7]:
# Not complete
train_data = tf.data.experimental.make_csv_dataset(train_csv_path, label_name="id"
                                                   batch_size=batch_size, header=True)

### Model 

In [20]:
# Defining model and loss

def BiRnn(text_batch, rating_batch, embedding_mat, reuse=False):
    # Define the container of the parameter
    Network = collections.namedtuple('Network', 'loss, pred_rating, \
                                                grads_and_vars, train, global_step, learning_rate')
    with tf.variable_scope("birnn", reuse=reuse):
        # embedding lookup
        embeddings = tf.nn.embedding_lookup(embedding_mat, text_batch)
        
        # Single bilstm layer
        forward_lstm_cell = tf.nn.rnn_cell.LSTMCell(NUM_HIDDEN)
        backward_lstm_cell = tf.nn.rnn_cell.LSTMCell(NUM_HIDDEN)
        
        (output_fw, output_bw), last_state = tf.nn.bidirectional_dynamic_rnn(forward_lstm_cell, 
                                                                             backward_lstm_cell, 
                                                                             embeddings)

        bidir_output = tf.concat([output_fw, output_bw], axis=2)
        
        # Fully connected
        logits = tf.contrib.layers.fully_connected(bidir_output, NUM_CLASSES, activation_fn=None)
        
        # prediction
        pred_probs = tf.nn.softmax(logits)
        
    loss =  tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=rating_batch, logits=logits), 
                          name= "loss")
    pred_rating = tf.argmax(pred_probs, 1)

    with tf.variable_scope("global_step_and_learning_rate", reuse=reuse):
        global_step = tf.contrib.framework.get_or_create_global_step()
        learning_rate = tf.train.exponential_decay(learning_rate, global_step, decay_steps=10000,
                                                   decay_rate = 0.1,
                                                   staircase=True)
        incr_global_step = tf.assign(global_step, global_step + 1)

    with tf.variable_scope("optimizer", reuse=reuse):
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            tvars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='birnn')
            optimizer = tf.train.AdamOptimizer(learning_rate, beta1=0.9)
            grads_and_vars = optimizer.compute_gradients(total_loss, tvars)
            train_op = optimizer.apply_gradients(grads_and_vars)

    return Network(
        loss = loss,
        pred_rating = pred_rating,
        grads_and_vars=grads_and_vars,
        train=tf.group(total_loss, incr_global_step, train_op),
        global_step=global_step,
        learning_rate=learning_rate
    )
        
        

In [ ]:
# Embedding matrix (Should be intialized with fasttext)
embedding_mat = np.random.rand(vocab_size,emb_size)

# Dummy input 
dummy_input_indices = np.random.randint(vocab_size, size=(batch_size, 10))
dummy_labels = np.random.randint(1, high=6, size=(batch_size,))

# Slightly bad implemenataion,
# need to take care of two scoper("birnn, embeddings") while saving and loading model
# For now keeping it non-trainable
with tf.variable_scope("embeddings"):
    embadding_mat = tf.Variable(tf.constant(0.0, shape=[vocab_size, emb_size]),
                    trainable=False, name="W")

    embedding_placeholder = tf.placeholder(tf.float32, [vocab_size, emb_size])
    embedding_init = embadding_mat.assign(embedding_placeholder)

# input placeholder
text_placeholder = tf.placeholder(tf.int32, shape=[batch_size, None], name="input_text")
rating_placeholder = tf.placeholder(tf.int8, shape=[batch_size,], name="input_labels")


# building network
net = BiRnn(text_placeholder, rating_placeholder, embadding_mat)
print('Finish building the birnn !!!')

# Start the session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Use supervisor to coordinate all queue and summary writer
# Deprecated, Update with tf.train.MonitoredTrainingSession
sv = tf.train.Supervisor(logdir=summary_dir, save_summaries_secs=0, saver=None)

# Training
with sv.managed_session(config=config) as sess:
    for step in range(max_iter):
            fetches = {
                "train": net.train,
                "global_step": sv.global_step,
                "loss": net.loss,
                "learning_rate": net.learning_rate
            }
            
            result = sess.run(fetches, feed_dict={embedding_placeholder: embedding_mat,
                                                 text_placeholder:dummy_input_indices,
                                                 rating_placeholder:dummy_labels})
            print("global_step: ", results["global_step"])
            print("learning_rate: ", results['learning_rate'])
            print("current_loss: ", results['loss'])
            print("-------------------------------------")